Саммаризация.

In [ ]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12 langchain-text-splitters==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.6 MB/s eta 0:00:00


In [ ]:
# импортируем необходимые библиотеки
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.docstore.document import Document
import os
import re
import requests
import tiktoken
import matplotlib.pyplot as plt
import textwrap
from textwrap import fill

import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR)

In [ ]:
# использование секретного ключа в колабе
from google.colab import userdata
# Получение API ключа из пользовательских данных Colab и установка его как переменной среды
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI(api_key=key)


Делаем функцию, которая будет саммаризировать диалог по мере его накапливания, и данную саммаризацию мы будем подавать модели, которая отвечает на вопрос клиента, чтобы модель учитывала контекст диалога.

## СУММАРИЗАЦИЯ

In [ ]:
prompt = '''
        Ты — нейро-саммаризатор. Твоя задача заключается в том, чтобы кратко суммировать диалог,
        представленный тебе. Обратите особое внимание на следующие моменты:\n\n
        1. Если клиент или менеджер упоминает своё имя, не указывай его в итоговом резюме.\n
        2. Суммаризация должна быть ясной и лаконичной, отражая основные моменты диалога.\n
        3. Соблюдай формальный и информативный стиль, чтобы итоговая версия была легкой для понимания.\n\n
     '''

In [ ]:
import openai
def summarize_questions(dialog):
    """
    Функция возвращает саммаризированный текст диалога.
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": "Саммаризируй следующий диалог консультанта и пользователя, удали из саммаризации имена клиента и менеджера: " + " ".join(dialog)}
    ]

    response=client.chat.completions.create(
        model="gpt-4o-mini",     # используем gpt4 для более точной саммаризации
        messages=messages,
        temperature=0,          # Используем более низкую температуру для более определенной суммаризации
    )

    return response.choices[0].message.content

In [ ]:
text = ''' Менеджер: Добрый день, интернет-магазин Tefal. Меня зовут Лев. Какой у вас вопрос?
Клиент: Алло, здравствуйте, меня зовут Мария. У меня там один заказик завис. Ну, в ожидании поступления, могу ли я его отменить и сделать возврат?
Менеджер: Так, а номер заказа есть у вас?
Клиент: Да, номер...
Менеджер: Термос, чайник, правильно?
Клиент: Ага.
Менеджер: Так, сейчас проверил, они есть в наличии и сможем вам их отправить.
Клиент: Угу.
Менеджер: Актуально еще будет?
Клиент: Да, наверное, нет. Давайте сделаем возврат.
Менеджер: Так, хорошо. Тогда сейчас я фиксирую информацию, отменим данный заказ, денежные средства вернутся в течение 10 календарных дней с этого момента.
Клиент: Спасибо большое.
Менеджер: Остались у вас еще вопросы?
Клиент: Нет, спасибо.
Менеджер: Благодарю вас за обращение. Всего доброго. До свидания.
Клиент: Да, всего доброго.
'''
summarize_questions(text)

'Консультант приветствует клиента и спрашивает о его вопросе. Клиент сообщает о заказе, который он хочет отменить и вернуть. Консультант уточняет номер заказа и подтверждает наличие товара. Клиент соглашается на возврат. Консультант информирует, что информация будет зафиксирована, и деньги вернутся в течение 10 календарных дней. Клиент благодарит и подтверждает, что у него больше нет вопросов. Консультант прощается.'

In [ ]:
text = '''
Клиент: Добрый день, я хотела бы спросить насчет кофемашинок.
Менеджер: Так, слушаю, какой вопрос у вас?
Клиент: Мы хотим купить в офис кофемашинку, у нас там будет 15 человек, ну и как-то просто подобрать хорошую, но не очень дорогую.
Менеджер:Так, ну все кофемашины представлены у нас, они от одного производителя, по качеству они не будут уступать друг другу, тут разница только в функциональных возможностях и, соответственно, в цене.
Клиент: Угу, так, то есть они просто ничем почти не отличаются?
Менеджер: Ну, у более дорогих моделей будет больше рецептов, удобнее управление, сенсорный экран у последних моделей сейчас появился. Так, именно по приготовлению кофе, если стандартные рецепты нужны, то можете более бюджетные модели рассматривать.
Клиент: Угу, поняла, спасибо.
Менеджер:Благодарю вас за обращение, всего доброго, до свидания.
'''
summarize_questions(text)

'Клиент интересуется покупкой кофемашины для офиса, где будет 15 человек, и хочет выбрать хорошую, но недорогую модель. Менеджер объясняет, что все кофемашины от одного производителя и различаются только функциональными возможностями и ценой. Более дорогие модели предлагают больше рецептов и удобное управление, включая сенсорный экран. Если нужны стандартные рецепты, можно рассмотреть более бюджетные варианты. Клиент благодарит за информацию.'

In [ ]:
text = '''
Менеджер: Добрый день, Ханцентромагазин (скорее всего название не такое, непонятно что он говорит), меня Виталий зовут. Как к вам обращаться?
Клиент: Здрасте. Подскажите, пожалуйста, меня интересует встраиваемая посудомоечная машина, ширина стандартная 60, а вот глубина, чтобы была не 550, а 500. Есть такие модели?
Менеджер: Сейчас проверю. Так, встраиваемая посудомоечная машина, да, вы сказали?
Клиент: Да, да, да.
Менеджер: Сейчас проверим. Ну, у нас на сегодняшний день вообще ни одной встраиваемой посудомоечной машины нет в продаже в интернет-магазине. А по поводу глубины я сейчас подскажу, они у нас все стандартные.
550, да?
Менеджер: Сейчас проверю. Ну, да, они все на 55, которые на 60 ширина.
Клиент: Ага, ага. Ясно, спасибо.
Менеджер: Чем-то еще могу помочь вам?
Клиент: Нет, спасибо.
Менеджер: И вам спасибо за обращение, Антон. Хорошего вам дня.
Менеджер: Оставайтесь на линии для оценки нашего (непонятное слово). Всего доброго. До свидания.
'''
summarize_questions(text)

'Консультант приветствует клиента и предлагает помощь. Клиент интересуется встроенной посудомоечной машиной с шириной 60 см и глубиной 500 мм. Консультант проверяет наличие такой модели, но сообщает, что в интернет-магазине нет встроенных посудомоечных машин с указанной глубиной. Все доступные модели имеют стандартную глубину 550 мм. Клиент благодарит за информацию, и консультант предлагает дополнительную помощь, на что клиент отвечает, что больше вопросов нет. В завершение консультант желает клиенту хорошего дня.'